# Acquisition Data Preparation: Incomplete Deals
Here we obtain data for deals which were proposed but dismissed.

In [1]:
import lseg.data as ld
from lseg.data.content import search
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, HTML
import re

## Search for all Withdrawn M&A Bids

In [13]:
ld.open_session()

deals = []

to_select = [
    "AcquirerName",
    "TargetName",
    "TransactionAnnouncementDate",
    "PercentageOfSharesAcquired",
    "AcquirerRegion",
    "TargetRegion",
    "PricePerShare",
    "CurrencyCodeOfTransaction",
    "DealPurpose",
    "FormOfTransactionName",
    "TransactionStatus",
    "InitialPricePerShare",
    "PremiumAsOn1DayPriorToAnnouncementDate",
    "PremiumAsOn1WeekPriorToAnnouncementDate",
    "PremiumAsOn4WeeksPriorToAnnouncementDate",
    "AcquirerPublicStatus",
    "TargetPublicStatus",
    "AcquirerMidIndustry",
    "TargetMidIndustry",
    "AnalystEstimatedValue",
    "RatioOfTransactionValueToNetIncome",
    "RatioOfTransactionValueToCashFlow",
    "RatioOfTransactionValueToSales",
    "RatioOfRankvalueToNetIncome",
    "RatioOfRankvalueToCashFlow",
    "RatioOfRankvalueToSales",
    "TransactionValueIncludingNetDebtOfTarget"
]

select_string = ", ".join(to_select)

deals_df = ld.discovery.search(
        view=search.Views.DEALS_MERGERS_AND_ACQUISITIONS)

deals_df


print(len(deals_df))



10


## Filtering
Need to ensure certain conditions are meant in order to be able to label the deal. Specifically, we need to ensure the transaction was completed, valuation data is accessible, and the deal happened at least three years ago so there is sufficient time to asess target performance after the deal was proposed. 

In [10]:
# Convert 'TransactionEffectiveDate' to datetime if it's not already
deals_df['TransactionEffectiveDate'] = pd.to_datetime(deals_df['TransactionAnnouncementDate'], errors='coerce')

# Calculate the date three years prior to today
three_years_prior = pd.Timestamp(datetime.now()) - pd.DateOffset(years=3)

# Filter the DataFrame based on the conditions
filtered_deals_df = deals_df[
    ((deals_df['PricePerShare'].notna()) & (deals_df['PricePerShare'] != 0.0)) &
    (deals_df['TransactionEffectiveDate'] < three_years_prior) & 
    (deals_df['TransactionEffectiveDate'].notna())
]

# Display the length of the filtered DataFrame and the first few rows
print(f"Length of flitered_deals_df: {len(filtered_deals_df)}")
filtered_deals_df.head()

Length of flitered_deals_df: 8


,AcquirerName,TargetName,TransactionAnnouncementDate,PercentageOfSharesAcquired,AcquirerRegion,TargetRegion,PricePerShare,CurrencyCodeOfTransaction,DealPurpose,FormOfTransactionName,...,TargetMidIndustry,AnalystEstimatedValue,RatioOfTransactionValueToNetIncome,RatioOfTransactionValueToCashFlow,RatioOfTransactionValueToSales,RatioOfRankvalueToNetIncome,RatioOfRankvalueToCashFlow,RatioOfRankvalueToSales,TransactionValueIncludingNetDebtOfTarget,TransactionEffectiveDate
1,[Comcast Corp],[TFCF Corp],2018-06-13,0,Americas,Americas,35.0,US,"[Sell a loss making/bankrupt operation, Streng...",Merger,...,Cable,<NA>,21.308,10.182,2.22,25.457,12.165,2.652,77464.372,2018-06-13
2,[BHP Group Ltd],[Anglo American PLC],2024-04-25,0,Asia Pacific Excluding Central Asia,Europe,27.183,STG,[Strengthen operations],Merger,...,Metals & Mining,<NA>,27.1,3.907,1.188,35.514,5.12,1.557,47648.393,2024-04-25
3,[Chevron Corp],[Anadarko Petroleum Corp],2019-04-12,0,Americas,Americas,64.996,US,"[Create synergies, eliminate duplicate service...",Merger,...,Oil & Gas,<NA>,62.177,6.162,3.631,63.345,6.277,3.699,48712.296,2019-04-12
4,[NVIDIA Corp],[ARM Ltd],2020-09-13,0,Americas,Europe,<NA>,STG,"[Create synergies, eliminate duplicate service...",Merger,...,Semiconductors & Semiconductor Equipment,32438.668956,-497.456,20.504,20.963,-483.723,19.938,20.385,40000.0,2020-09-13
6,[Hong Kong Exchanges and Clearing Ltd],[London Stock Exchange Group PLC],2019-09-11,0,Asia Pacific Excluding Central Asia,Europe,103.287,STG,"[Expand presence in new/foreign markets, Creat...",Merger,...,Brokerage,<NA>,110.412,51.97,25.666,110.412,51.97,25.666,37259.153,2019-09-11


## Download the Dataset
The next step from here is to label the dataset in retroactive_dcf.ipynb and acquisition_labeling.ipynb

In [201]:
csv_filename = 'incomplete_acquisition_deals.csv'
filtered_deals_df.to_csv(csv_filename, index=False)

print(f'Data saved to {csv_filename}')

Data saved to executed_acquisition_deals.csv
